In [2]:
import scanpy as sc
import numpy as np
import warnings
warnings.filterwarnings("ignore")

from scimilarity.utils import lognorm_counts, align_dataset, get_centroid
from scimilarity import CellEmbedding, CellQuery
import os
random_seed = 42
np.random.seed(random_seed)

import pandas as pd

sc.set_figure_params(dpi=100)

In [ ]:
# Instantiate the CellQuery object
# Set model_path to the location of the uncompressed model
model_path = "/home/nicolas/Documents/ai-center-project/models/model_v1.1"
cq = CellQuery(model_path)

In [3]:
data_path = "/home/nicolas/Downloads/NB.bone.Met_preprocessed.h5ad"
data = sc.read(data_path)
print(f"Data cell 1: {data.obs['cell1']}")

#subset = data[data.obs['cell1'].isin(["B cells", "T cells", "myeloid", "immature B", "Neutrophil", "progenitor"])]


Data cell 1: cell_ID
NBBM1_AAACGAAGTCCGGCAT-1           T cells
NBBM1_AAACGCTGTCACAATC-1           T cells
NBBM1_AAAGGATGTAGGCTGA-1           B cells
NBBM1_AAAGGTAGTCTCAGGC-1           B cells
NBBM1_AAAGGTAGTGAGTGAC-1           T cells
                                   ...    
NB34_Bone_TTTGTTGAGCGAAACC-1       T cells
NB34_Bone_TTTGTTGAGTCGCTAT-1    Neutrophil
NB34_Bone_TTTGTTGAGTCGGCAA-1       T cells
NB34_Bone_TTTGTTGGTAAGGCTG-1       T cells
NB34_Bone_TTTGTTGGTGCGTGCT-1       B cells
Name: cell1, Length: 36763, dtype: category
Categories (13, object): ['B cells', 'NK', 'Neutrophil', 'T cells', ..., 'plasma', 'pro B', 'proerythroblast', 'progenitor']


In [ ]:
data.X = data.layers['RNA']
data.layers['counts']=data.layers['RNA']

data = align_dataset(data, cq.gene_order)
data = lognorm_counts(data)

In [ ]:
fm_basic_signature = ["B cells", "T cells", "myeloid", "immature B", "Neutrophil", "progenitor"]
sc.tl.score_genes(data, fm_basic_signature)

In [ ]:
sig_query_threshold = data.obs.score.quantile(0.999)
cells_used_in_query = data.obs.score >= sig_query_threshold
data.obs["used_in_query"] = cells_used_in_query

In [ ]:
avg_cell = get_centroid(data.layers["counts"][data.obs["used_in_query"]])
avg_embedding = cq.get_embeddings(avg_cell)